In [4]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import rootpy
import scipy

from FluorSpec.PTI_Data import PTI_Data

In [19]:
file_names = ["Noah/Integrating Sphere Tests for PPO Contamination/EmissionScan_NoCuvette_300_290-450_0point5nm_2sec_FS_20170613_1138.txt"]

fs_data = PTI_Data(file_names[0])
print fs_data

Initializing PTI_Data at Tue Jun 13 16:59:33 2017
<FluorSpec.PTI_Data.PTI_Data instance at 0x7ff126439638>
